# Introduzione


# Librerie
Allo scopo di migliorare, risolvendo i bug incontrati, e potenziare i meccanismi per la risoluzione del 3D bin packing ho sviluppato la seguente libreria:

In [ ]:
import py3dbl

# Elementi per casi di studio
Per eseguire esperimenti su un caso realistico consideriamo un mezzo atto allo scopo della logistica dell'ultimo miglio, in particolare un Fiat Ducato:

In [ ]:
from decimal import Decimal

@py3dbl.constraint(11)
def can_pass_over_the_upper_wheel(bin : py3dbl.Bin, item : py3dbl.Item):
    return item.position.z > Decimal(2.8) or (item.width < Decimal(1.442) or item.height < Decimal(1.432))

constraints=[
    py3dbl.constraints['weight_within_limit'],
    py3dbl.constraints['fits_inside_bin'],
    py3dbl.constraints['no_overlap'],
    py3dbl.constraints['is_supported'],
    py3dbl.constraints['can_pass_over_the_upper_wheel']
]

DucatoL2H2 = py3dbl.BinModel(
    name = "DucatoL2H2",
    size = (1.87,1.932,3.120),
    max_weight = 3000,
    constraints = constraints,
    dead_volumes = [
        py3dbl.Volume([.024,.4,.5],[0,0,2.30]),py3dbl.Volume([.15,.38,.48],[.024,0,2.32]),py3dbl.Volume([.05,.36,.47],[.024+0.15,0,2.33]),
        py3dbl.Volume([.024,.4,.5],[1.87-.024,0,2.30]),py3dbl.Volume([.15,.38,.48],[1.87-.15-.024,0,2.32]),py3dbl.Volume([.05,.36,.48],[1.87-.05-.15-.024,0,2.33])
    ]
)

Generiamo una lista di ordini:

In [ ]:
orders = py3dbl.item_generator(
    width  = (0.05,1.5),
    height = (0.05,1.5),
    depth  = (0.05,1.5),
    weight = (10,1),
    priority_range = (0,10),
    batch_size   = 400,
    use_gaussian_distrib = False
)

In [ ]:
py3dbl.constraints['is_supported'].set_parameter("allow_item_fall",True)
py3dbl.constraints['is_supported'].set_parameter("minimum_support",0.9)
py3dbl.algorithms['all_stand'].set_parameter("allow_full_rotation",False)

Configuriamo il risolutore (Packer)

In [ ]:
packer = py3dbl.Packer()
packer.set_default_bin(DucatoL2H2)
packer.add_batch(orders)

Lanciamo la risoluzione:

In [ ]:
import time
start = time.time()
packer.pack(
    algorithm=py3dbl.algorithms['all_stand']
)
end = time.time()
print("Time: ",end - start)

In [ ]:
print(packer.calculate_statistics())
print(len(packer.current_configuration))
print(len(packer.items))
acc = 0
py3dbl.render_bin_interactive(packer.current_configuration[0],render_bin=False,border_width=1.0,transparency=.8)
for bin in packer.current_configuration:
    #py3dbl.render_bin_interactive(bin,render_bin=True,border_width=1.0,transparency=.8)
    acc += len(bin.items)
print(acc)

Vediamo i risultati: